# [ BERT - 잡코리아 기업평 긍정/부정 분류 ]

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, Dropout, LSTM, MaxPooling1D, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from keras.layers.embeddings import Embedding
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping

import re
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/BOAZ 스터디/nlp 스터디/train.csv')
test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/BOAZ 스터디/nlp 스터디/test.csv')
sub = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/BOAZ 스터디/nlp 스터디/sample_submission.csv')

In [6]:
print('Train Data: ', train.shape)
print('Test Data: ', test.shape)

Train Data:  (65000, 10)
Test Data:  (20934, 9)


In [7]:
# 간단한 불필요 문자, 단어들을 지워주었습니다.
def cleaning_text(x):
    x = re.sub('[^\w\s]', '', x)
    x = re.sub('[\rob\r]', '', x)
    return re.sub('[ㄱ-ㅎㅏ-ㅣ]', '', x)

train['기업한줄평'] = train['기업한줄평'].apply(cleaning_text)
test['기업한줄평'] = test['기업한줄평'].apply(cleaning_text)

In [9]:
!pip install transformers

     |████████████████████████████████| 1.8MB 13.5MB/s 
     |████████████████████████████████| 3.2MB 51.7MB/s 
     |████████████████████████████████| 890kB 57.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=f643ea849b300745637d4a07b4f3437794c0453eed8e2561d04b0b46424607ca
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [10]:
import torch
from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import random
import time
import datetime

In [11]:
import os

n_devices = torch.cuda.device_count()
print(n_devices)

for i in range(n_devices):
    print(torch.cuda.get_device_name(i))  # gpu 버전 확인

1
Tesla V100-SXM2-16GB


In [14]:
display(train.head(2))
display(test.head(2))

,ID,기업명,직무,재직상태,작성일자,기업한줄평,평점,기업추천율(%),CEO지지율(%),기업성장률(%)
0,0,(주)코스트코코리아,서비스/고객지원,현직원,2020.07,절대가지마세요 몸 다 상합니다 부서 절대요,0,44,47,37
1,1,(주)비상교육,교육,현직원,2020.12,욕심없는 사람에게 추천 도태되는 게 조금이라도 두려운 사람에겐 비추,0,54,45,16


,ID,기업명,직무,재직상태,작성일자,기업한줄평,기업추천율(%),CEO지지율(%),기업성장률(%)
0,0,(주)크라운제과,생산관리/품질관리,전직원,2020.05,그럭저럭 괜찮지만 연봉이 낮다 나이는 낮아서 잘 적응하면 다닐만 할거같다,16,15,8
1,1,국립암센터,연구개발,전직원,2019.09,낮은임금 아무리 뛰어난 인재도 감당 못하는곳,26,29,8


In [15]:
# bert의 앞 뒤 [CLS], [SEP] 넣어주기
document_bert = ["[CLS] " + str(s) + " [SEP]" for s in train.기업한줄평]  # train
document_bert[:5]

['[CLS] 절대가지마세요 몸 다 상합니다  부서 절대요 [SEP]',
 '[CLS] 욕심없는 사람에게 추천 도태되는 게 조금이라도 두려운 사람에겐 비추 [SEP]',
 '[CLS] 팀 간의 상하관계가 분명하고 상사는 부하직원에게 일을 몰아줌 [SEP]',
 '[CLS] 애경 그룹내 작지만 꾸준한 성장을 하는 회사소소한 복리후생이 있고약간은 보수적이나 지속적으로 직원을 위해노력하고 있다는 느낌을 주는회사 [SEP]',
 '[CLS] 다른 대기업보다 급여가 높으며 사원들도 인성좋고 착함 [SEP]']

In [16]:
# bert 토크나이저 이용
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
tokenized_texts = [tokenizer.tokenize(s) for s in document_bert]
print(tokenized_texts[0])


['[CLS]', '절', '##대가', '##지', '##마', '##세', '##요', '몸', '다', '상', '##합', '##니다', '부', '##서', '절', '##대', '##요', '[SEP]']


In [19]:
len(tokenized_texts)

65000

In [23]:
MAX_LEN = 128
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype='long', truncating='post', padding='post')
input_ids[0]

array([  101,  9666, 97048, 12508, 23811, 24982, 48549,  9288,  9056,
        9414, 33188, 48345,  9365, 12424,  9666, 14423, 48549,   102,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0]

In [24]:
# 어텐션마스크
attention_masks = []

for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)
    
print(attention_masks[0])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [25]:
train.head(2)

,ID,기업명,직무,재직상태,작성일자,기업한줄평,평점,기업추천율(%),CEO지지율(%),기업성장률(%)
0,0,(주)코스트코코리아,서비스/고객지원,현직원,2020.07,절대가지마세요 몸 다 상합니다 부서 절대요,0,44,47,37
1,1,(주)비상교육,교육,현직원,2020.12,욕심없는 사람에게 추천 도태되는 게 조금이라도 두려운 사람에겐 비추,0,54,45,16


In [26]:
# train/valid split
train_inputs, validation_inputs, train_labels, validation_labels = \
train_test_split(input_ids, train['평점'].values, random_state=42, test_size=0.1)

train_masks, validation_masks, _, _ = train_test_split(attention_masks, 
                                                       input_ids,
                                                       random_state=42, 
                                                       test_size=0.1)

In [27]:
# 파이토치 텐서로 변환
train_inputs = torch.tensor(train_inputs)
train_labels = torch.tensor(train_labels)
train_masks = torch.tensor(train_masks)
validation_inputs = torch.tensor(validation_inputs)
validation_labels = torch.tensor(validation_labels)
validation_masks = torch.tensor(validation_masks)

In [31]:
BATCH_SIZE = 32

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=BATCH_SIZE)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=BATCH_SIZE)

In [38]:
# 분류를 위한 bert모델 생성
model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=2)  # 이진분류니까 num_labels는 2
model.cuda()

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [39]:
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # learning rate
                  eps = 1e-8 # 0으로 나누는 것을 방지하기 위한 epsilon 값
                )

# 에폭수
epochs = 4

# 총 훈련 스텝
total_steps = len(train_dataloader) * epochs

# lr 조금씩 감소시키는 스케줄러
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

In [40]:
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

# 시간 표시 함수
def format_time(elapsed):
    # 반올림
    elapsed_rounded = int(round((elapsed)))
    # hh:mm:ss으로 형태 변경
    return str(datetime.timedelta(seconds=elapsed_rounded))

# Train

In [42]:
%%time
seed_val = 42   # 재현을 위해 랜덤시드 고정
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# 그래디언트 초기화
model.zero_grad()

# 에폭만큼 반복
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # 시작 시간 설정
    t0 = time.time()

    # 로스 초기화
    total_loss = 0

    # 훈련모드로 변경
    model.train()
        
    # 데이터로더에서 배치만큼 반복하여 가져옴
    for step, batch in enumerate(train_dataloader):
        # 경과 정보 표시
        if step % 500 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch

        # Forward 수행                
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)
        
        # 로스 구함
        loss = outputs[0]

        # 총 로스 계산
        total_loss += loss.item()

        # Backward 수행으로 그래디언트 계산
        loss.backward()

        # 그래디언트 클리핑
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # 그래디언트를 통해 가중치 파라미터 업데이트
        optimizer.step()

        # 스케줄러로 학습률 감소
        scheduler.step()

        # 그래디언트 초기화
        model.zero_grad()

    # 평균 로스 계산
    avg_train_loss = total_loss / len(train_dataloader)            

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")

    #시작 시간 설정
    t0 = time.time()

    # 평가모드로 변경
    model.eval()

    # 변수 초기화
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # 데이터로더에서 배치만큼 반복하여 가져옴
    for batch in validation_dataloader:
        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch
        
        # 그래디언트 계산 안함
        with torch.no_grad():     
            # Forward 수행
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        # 로스 구함
        logits = outputs[0]

        # CPU로 데이터 이동
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # 출력 로짓과 라벨을 비교하여 정확도 계산
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")


======== Epoch 1 / 4 ========
Training...
  Batch   500  of  1,829.    Elapsed: 0:01:56.
  Batch 1,000  of  1,829.    Elapsed: 0:03:52.
  Batch 1,500  of  1,829.    Elapsed: 0:05:47.

  Average training loss: 0.45
  Training epcoh took: 0:07:03

Running Validation...
  Accuracy: 0.82
  Validation took: 0:00:13

======== Epoch 2 / 4 ========
Training...
  Batch   500  of  1,829.    Elapsed: 0:01:55.
  Batch 1,000  of  1,829.    Elapsed: 0:03:50.
  Batch 1,500  of  1,829.    Elapsed: 0:05:45.

  Average training loss: 0.36
  Training epcoh took: 0:07:01

Running Validation...
  Accuracy: 0.83
  Validation took: 0:00:13

======== Epoch 3 / 4 ========
Training...
  Batch   500  of  1,829.    Elapsed: 0:01:55.
  Batch 1,000  of  1,829.    Elapsed: 0:03:50.
  Batch 1,500  of  1,829.    Elapsed: 0:05:45.

  Average training loss: 0.30
  Training epcoh took: 0:07:00

Running Validation...
  Accuracy: 0.83
  Validation took: 0:00:13

======== Epoch 4 / 4 ========
Training...
  Batch   500  of 

# Test

In [103]:
# 문장 테스트
# 입력 데이터 변환
def convert_input_data(sentences):

    # BERT의 토크나이저로 문장을 토큰으로 분리
    tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

    # 입력 토큰의 최대 시퀀스 길이
    MAX_LEN = 128

    # 토큰을 숫자 인덱스로 변환
    input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
    
    # 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
    input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

    # 어텐션 마스크 초기화
    attention_masks = []

    # 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
    # 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
    for seq in input_ids:
        seq_mask = [float(i>0) for i in seq]
        attention_masks.append(seq_mask)

    # 데이터를 파이토치의 텐서로 변환
    inputs = torch.tensor(input_ids)
    masks = torch.tensor(attention_masks)

    return inputs, masks
    
def test_sentences(sentences):

    # 평가모드로 변경
    model.eval()

    # 문장을 입력 데이터로 변환
    inputs, masks = convert_input_data(sentences)

    # 데이터를 GPU에 넣음
    b_input_ids = inputs.to(device)
    b_input_mask = masks.to(device)
            
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)

    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()

    return logits

In [104]:
test.head()

,ID,기업명,직무,재직상태,작성일자,기업한줄평,기업추천율(%),CEO지지율(%),기업성장률(%),평점
0,0,(주)크라운제과,생산관리/품질관리,전직원,2020.05,그럭저럭 괜찮지만 연봉이 낮다 나이는 낮아서 잘 적응하면 다닐만 할거같다,16,15,8,0
1,1,국립암센터,연구개발,전직원,2019.09,낮은임금 아무리 뛰어난 인재도 감당 못하는곳,26,29,8,0
2,2,엔에이치엔(주),인사/총무,전직원,2020.03,근무환경이 너무나도 좋은곳 게임개발회사로 입지가 탄탄,63,42,30,0
3,3,엘아이지넥스원(주),경영/기획/컨설팅,현직원,2019.04,자유로운 환경에서 근무할 수 있습니다,59,37,27,0
4,4,(주)세코닉스,연구개발,전직원,2017.11,생산업체일뿐 사실상 개발하는 게 없음사장님 말에 모든게 좌우됨복지는 기숙사90년대 ...,20,24,18,0


In [108]:
test_list = test['기업한줄평'].tolist()

In [126]:
len(test_list)

20934

In [109]:
test_list[0]

'그럭저럭 괜찮지만 연봉이 낮다 나이는 낮아서 잘 적응하면 다닐만 할거같다'

In [121]:
np.argmax(test_sentences(['근무환경이 너무나도 좋은곳 게임개발회사로 입지가 탄탄']))

1

In [123]:
[test_list[0]]

['그럭저럭 괜찮지만 연봉이 낮다 나이는 낮아서 잘 적응하면 다닐만 할거같다']

In [128]:
%%time
from tqdm.notebook import tqdm 
score = []
for i in tqdm(range(len(test_list))):
  logits = np.argmax(test_sentences([test_list[i]]))
  score.append(logits)


CPU times: user 4min 38s, sys: 1.43 s, total: 4min 40s
Wall time: 4min 39s


In [129]:
score

[0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,


In [130]:
final_sub = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/BOAZ 스터디/nlp 스터디/sample_submission.csv')
final_sub['평점'] = score
final_sub.to_csv('/content/drive/MyDrive/Colab Notebooks/BOAZ 스터디/nlp 스터디/sub/0216_b.csv', index = False)